In [1]:
import schedule
import pygame
import time

pygame 2.0.0 (SDL 2.0.12, python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
key = 'urz15HMZIl7GH7VC1sxsnGwRWNAHv%2Bfo9bhWTmlP3P9i3tELJ2oomcBTQtNZBIsxXi5PmFAcw1Yajfb3xvGPsA%3D%3D'

In [3]:
from datetime import datetime as dt
today = dt.today()
date = "%04d%02d%02d"%(today.year, today.month, today.day)

In [4]:
today_time = '0200'

In [5]:
url_api = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?serviceKey={}&numOfRows=100&pageNo=1&base_date={}&base_time={}&nx=58&ny=74&dataType=JSON'.format(key,date,today_time)

In [6]:
import requests
resp = requests.get(url_api)

In [7]:
data = resp.json()['response']['body']['items']['item']

In [8]:
import pandas as pd

In [9]:
data_df = pd.DataFrame(data)

In [10]:
data_df

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20201215,0200,POP,20201215,0600,80,58,74
1,20201215,0200,PTY,20201215,0600,3,58,74
2,20201215,0200,R06,20201215,0600,1.5,58,74
3,20201215,0200,REH,20201215,0600,70,58,74
4,20201215,0200,S06,20201215,0600,1.2,58,74
...,...,...,...,...,...,...,...,...
95,20201215,0200,PTY,20201216,0900,0,58,74
96,20201215,0200,REH,20201216,0900,65,58,74
97,20201215,0200,SKY,20201216,0900,4,58,74
98,20201215,0200,T3H,20201216,0900,-4,58,74


In [11]:
data_df.loc[:,['category','fcstValue']]

,category,fcstValue
0,POP,80
1,PTY,3
2,R06,1.5
3,REH,70
4,S06,1.2
...,...,...
95,PTY,0
96,REH,65
97,SKY,4
98,T3H,-4


In [12]:
data1 = data_df.loc[:,["fcstDate",'fcstTime',"category","fcstValue"]]

In [13]:
data1[(data1['category']=='POP') | (data1['category']=='SKY') | (data1['category']=='REH') 
     | (data1['category']=='T3H')|(data1['category']=='TMN')|(data1['category']=='TMX')]

,fcstDate,fcstTime,category,fcstValue
0,20201215,0600,POP,80
3,20201215,0600,REH,70
5,20201215,0600,SKY,4
6,20201215,0600,T3H,-5
7,20201215,0600,TMN,-5.0
12,20201215,0900,POP,80
14,20201215,0900,REH,60
15,20201215,0900,SKY,4
16,20201215,0900,T3H,-5
21,20201215,1200,POP,30


In [14]:
data1.columns = ["일자","시간","정보","측정값"]

In [15]:
data1.loc[data1['정보']== 'POP', '정보'] = '강수확률'
data1.loc[data1['정보']== 'SKY', '정보'] = '하늘상태'
data1.loc[data1['정보']== 'TMN', '정보'] = '아침 최저기온'
data1.loc[data1['정보']== 'TMX', '정보'] = '낮 최고기온'

In [16]:
data2 = data1[data1['일자']==date]

In [17]:
data3 = data2[(data2['정보']=='강수확률') | (data2['정보']=='하늘상태') |(data2['정보']=='아침 최저기온')|(data2['정보']=='낮 최고기온')]

In [18]:
df2 = data2[data2['정보'] == '강수확률']

In [19]:
df3 = data2[(data2['정보'] == '아침 최저기온') | (data2['정보'] == '낮 최고기온')]

In [20]:
print(df3)

          일자    시간       정보   측정값
7   20201215  0600  아침 최저기온  -5.0
37  20201215  1500   낮 최고기온   0.0


In [21]:
def weather(row):
    if row['정보']=='하늘상태':
        if row['측정값'] == '1':
            row['측정값'] = "맑음"
            return row
        if row['측정값'] == '3':
            row['측정값'] = "구름 많음"
            return row
        if row['측정값'] == '4':
            row['측정값'] = "흐림"
            return row
    else:
        return row


In [22]:
#df['측정값'] = weather_value.apply(weather)
data3 = data3.apply(weather, axis=1)

In [23]:
df = data3[data3['정보'] == '하늘상태']

In [24]:
data3.index=range(len(data3.index))

In [25]:
import requests

In [26]:
f = open("kakaokey.txt", 'r')
SECRET_KEY =f.readline()
f.close()
SECRET_KEY

'94eff828eb6d4a0d102161d583d69e79'

In [27]:
# Python 파일
import urllib.request

url = 'https://kakaoi-newtone-openapi.kakao.com/v1/synthesize'
request = urllib.request.Request(url)
request.add_header('Host','kakaoi-newtone-openapi.kakao.com')
request.add_header('Content-Type','application/xml')
request.add_header('Authorization',f'KakaoAK {SECRET_KEY}')

#VoiceName = 'WOMAN_DIALOG_BRIGHT'
#text = "할머니! 오늘은 날씨가 정말 좋아요!! 기분 좋은 하루 보내세요."
#data = "<speak><voice name='" + VoiceName + "'>" + text + "</voice></speak>"
#response = urllib.request.urlopen(request, data = data.encode('utf-8'))
# 요청 URL과 headers, data를 post방식으로 보내준다.

In [28]:
def Tweather():
    VoiceName = 'WOMAN_DIALOG_BRIGHT'
    if df['측정값'].iloc[0] == '맑음':  
        text = "할머니! 오늘은 날씨가 정말 좋아요!! 기분 좋은 하루 보내세요."

    elif df['측정값'].iloc[0] == '구름 많음':  
        text = "할머니! 오늘은 구름이 많아 날씨가 추울 수 있으니 옷을 따뜻하게 입으세요."
    
    elif df['측정값'].iloc[0] == '흐림':
        text = "할머니! 오늘은 강수확률{} 퍼센트로 비 가 올 수 있으니 우산을 꼭 챙겨 주세요!!.".format(df2['측정값'].iloc[0])
    
    data = "<speak><voice name='" + VoiceName + "'>" + text + "</voice></speak>"
    response = urllib.request.urlopen(request, data = data.encode('utf-8'))
    
    writeFile=open('test.mp3','wb')
    writeFile.write(response.read())
    writeFile.close()
    
    pygame.mixer.init()
    pygame.mixer.music.load("test.mp3")
    pygame.mixer.music.play()
    time.sleep(7) # 문장이 5초 이상 될 것같은 경우 sleep 시간 조절.
    pygame.mixer.quit()


In [29]:
def temperature():
    VoiceName = 'WOMAN_DIALOG_BRIGHT'
    text2 = "할머니! 오늘 아침 최저 기온은 {}도 이고, 낮 최고 기온은 {}도 입니다.".format(df3['측정값'].iloc[0],df3['측정값'].iloc[1])
    data = "<speak><voice name='" + VoiceName + "'>" + text2 + "</voice></speak>"
    response = urllib.request.urlopen(request, data = data.encode('utf-8'))

    writeFile=open('test2.mp3','wb')
    writeFile.write(response.read())
    writeFile.close()
    
    pygame.mixer.init()
    pygame.mixer.music.load("test2.mp3")
    pygame.mixer.music.play()
    time.sleep(7) # 문장이 5초 이상 될 것같은 경우 sleep 시간 조절.
    pygame.mixer.quit()

In [30]:
schedule.every().day.at("15:22").do(Tweather)
schedule.every().day.at("15:22:06").do(temperature)

Every 1 day at 15:22:06 do temperature() (last run: [never], next run: 2020-12-15 15:22:06)

In [ ]:
while True:
    schedule.run_pending()